# Get Photometry Tables from CDS and adding the Bibcodes for each filter

From the objects that we find in NED with Ne emission lines in MIR, now we are going to get their corresponding phot tables from [CDS (Vizier)](http://vizier.unistra.fr/vizier/sed/?-c=&-c.r=10&-c.u=arcsec). The code to download and check the VOTables from CDS is based in Andres Ramos's work (https://github.com/aframosp/AGNView). Also the part to add the bibcodes to the CDS photometry tables

To reach this aim, in the function **photCDS** using with the tools **request.get** of **[Python Request](https://docs.python-requests.org/en/latest/)** we search the photometry through the vizier url address defined before for each object of the SIMBAD sample (first notebook) and save it in a vot file. Then with the function **check_vot** we check that the content of the votable is OK. If not, the function using tools like **Session** and **Request** deletes the content-length to try to get again the votable.


In [1]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [2]:
from datetime import date
import time
import requests
import numpy as np

from astroquery.vizier import Vizier

from pathlib import Path
from requests import Request, Session

import astropy.units as u
from astropy.table import hstack, QTable, Table

In the next cell we define the paths and create the folders where we will save the data downloaded from the CDS:

In [3]:
path_raw='../Datos/Raw/'
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/' #In this path wi will save the photometry tables for each object
Path(path_phot_cds).mkdir(parents=True, exist_ok=True)
path_cds_vot = '../Datos/Work/Phot_Tables_CDS/VOT/' 
Path(path_cds_vot).mkdir(parents=True, exist_ok=True)

In [4]:
def photCDS(url,name,path):
    req = requests.get(url,
                       headers={'cache-control': 'private,max-age=0,no-cache',
                                "Pragma": "no-cache"})
    if req.status_code == 200:
        with open(path+name+'_CDS_votables.vot', 'wb') as f_vot:
            f_vot.write(req.content)
            f_vot.close()
    check_vot(url,name,path)
    return req

def check_vot(url,name,path):
    """Check that the content of the votable is ok"""
    vot_bad = True
    while vot_bad:
        try:
            cds_VOTable = Table.read(path+name+'_CDS_votables'+'.vot',
                                        format='votable')
            print('Votable checked')
            vot_bad = False
        except ValueError:
            print('Error in votable, deleting content-length')
            ses = Session()
            req_p = Request('POST', url)
            prepped = req_p.prepare()
            del prepped.headers['Content-Length']
            ses.send(prepped)
            photCDS(url,name,path)

def add_bibcode_cds(name,path):
    '''
    Adding the Bibcode to the CDS tables
    '''
    cds_votable = Table.read(path+name+'_CDS_votables'+'.vot',
                             format='votable')                           
    cds_votable['Bibcode'] = np.array(['Empty']*len(cds_votable),
                                         dtype='object')
    print('Searching for Bibcode')
    for tabindx, tabinfo in enumerate(cds_votable['_tabname']):
        try:
            time.sleep(0.001) #This is used to avoid being flag by the server
            name_cat = tabinfo.rpartition('/')[0]
            search = Vizier.query_constraints(catalog='METAcat', name=name_cat)
            cds_votable['Bibcode'][tabindx] = search[0][0]['bibcode']
        except IndexError:
            print('There is an error at ', tabindx, tabinfo)
    print('Adding Bibcode to ',name, ' votable')        
    return cds_votable

In [5]:
date_sample = 'Sep_02_2022'
date_sample_2 = 'May_29_2023'

In [6]:
NeIR_Sy_obj_t = Table.read(path_raw+'Obj_Sy_Ne-IR_'+date_sample+'.vot',format='votable')
NeIR_Sy_obj_t.show_in_notebook()

idx,Main_Id,Id_used_NED,RA,DEC,otype_txt,redshift,rvz_type,ids,otypes,cds_url,SMB_sample_index,NED_Ne_IR_info
0,3C 234.0,3C 234.0,150.45634473807,28.7858176354,Sy1,0.18494,z,Gaia DR3 743659018773360128|WISEA J100149.52+284708.9|2MASX J10014950+2847089|GRA B0959+28|GRA J1001+28|1Jy 0958+290|1Jy 0958+29|3C 234.0|3C 234|3CR 234|4C 29.35|7C 0958+2901|B2 0958+29|B2.2 0958+29|BWE 0958+2901|CSO 31|CTA 49|CTD 64|Cul 0958+290|Cul 0958+29|DA 280|DB 72|GB6 B0958+2901|INTREF 402|IRAS 09589+2901|IRAS F09589+2901|LEDA 139234|LHE 265|MITG J100147+2846|NRAO 343|OHIO L 200|QSO B0958+290|S2 0958+29|TXS 0958+290|UT 0958+290|VRO 29.09.03|WB 0958+2901|[ATI98] 095851+2910|[BDW2002] q0958+2901|[HB91] 0958+290|[W72] 0958+29|[WTW94] 0958+290|[GW2008] CoNFIG 64|87GB 095856.9+290154|[VV2000c] J100149.5+284710|[VV2003c] J100149.5+284710|[VV2006c] J100149.5+284710|[VV98c] J100149.5+284710|SDSS J100149.52+284708.9|[VV2010c] J100149.5+284710|WISE J100149.52+284709.0|AKARI-IRC-V1 J1001495+284708|EQ J1001+287|PBC J1001.8+2847|NVSS J100146+284656|7C 095855.89+290131.00|[CAB95] IRAS F09589+2901|VLSS J1001.8+2847|SWIFT J1001.8+2848|Gaia DR2 743659018773360128,*|AGN|G|IR|QSO|Rad|Sy1|X|blu|gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=150.45634473807%2028.7858176354&-c.rs=5,283.0,NeII|NeIII|NeV
1,2MASS J13000535+1632148,NVSS J130005+163212,195.02228295163,16.53743305218,Sy1,0.07993,z,Gaia DR3 3937375195178654720|TIC 88137340|2MASS J13000535+1632148|NVSS J130005+163212|2MASX J13000533+1632151|2MASSI J1300053+163214|2MASS J13000534+1632149|FIRST J130005.3+163214|[VV2003c] J130005.3+163214|[VV2006c] J130005.3+163214|[VV2010c] J130005.3+163214|PBC J1300.1+1636|SDSS J130005.34+163214.8|SDSS J130005.35+163214.8|LEDA 3800210|SWIFT J1300.1+1635|AKARI-IRC-V1 J1300053+163214|Gaia DR2 3937375195178654720,*|AGN|G|IR|Rad|Sy1|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=195.02228295163%2016.53743305218&-c.rs=5,428.0,NeII|NeIII
2,NGC 262,NGC 262,12.196426138730002,31.956956641230004,Sy2,0.015260233939595302,v,Gaia DR3 360927649734789248|LAMOST J004847.14+315725.1|IERS B0046+316|[KRL2007b] 7|2E 189|1AXG J004847+3157|2E 0046.0+3140|B2.1 0046+31|BWE 0046+3141|GB6 B0046+3140|INTREF 37|IRAS 00461+3141|IRAS F00460+3141|JVAS J0048+3157|JVAS B0046+316|LEDA 2855|MAPS-PP O-601-810121|MCG+05-03-008|MITG J004846+3156|Mrk 348|NGC 262|NPM1G +31.0015|NVSS J004847+315725|Gaia DR1 360927645439037056|UVQS J004847.14+315725.0|TXS 0046+316|UGC 499|WB 0046+3141|WN B0046.0+3141|XSS J00485+3203|Z 501-20|[BTW2003] J0048+3159|[DML87] 16|[HB91] 0046+316|2MASX J00484711+3157249|2XMM J004847.1+315724|1RXP J004847.4+315716|87GB 004606.1+314104|ICRF J004847.1+315725|MY 004604.0+314145.8|SWIFT J0048.8+3155|SWIFT J0048.8+3157|Z 0046.1+3142|[CHM2007] HDC 36 J004847.11+3157249|[CHM2007] LDC 43 J004847.11+3157249|[M98c] 004604.4+314100|[VV2000c] J004847.2+315725|[VV2003c] J004847.2+315725|[VV2006c] J004847.2+315725|[VV98c] J004847.2+315725|UZC J004847.2+315725|CGRaBS J0048+3157|PSCz Q00461+3141|[MGL2009] BZU J0048+3157|PBC J0048.7+3157|SWIFT J0048.7+3157|MAXI J0048+320|[VV2010c] J004847.2+315725|QSO J0048+3157|QSO B0046+316|[MML2015] 5BZG J0048+3157|[CAB95] IRAS F00460+3141|CLASS J004847.1435+315725.092|2MAXI J0048+320|Gaia DR2 360927649734789248,*|AGN|Bla|G|GiG|IR|QSO|Rad|Sy2|X|gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=12.196426138730002%2031.956956641230004&-c.rs=5,484.0,NeII|NeIII|NeV
3,IRAS 04385-0828,IRAS 04385-0828,70.2290037068,-8.37281297812,Sy2,0.015064,z,Gaia DR3 3186063094441390720|6dFGS gJ044055.0-082222|IRAS F04385-0828|IRAS 04385-0828|LEDA 15806|NPM1G -08.0177|[DML87] 141|[HB91] 0438-084|[VFT93] 0438-0828|[VV2000c] J044055.0-082222|[VV2003c] J044055.0-082222|[VV2006c] J044055.0-082222|2MASX J04405494-0822221|[CHM2007] HDC 310 J044054.94-0822221|[CHM2007] LDC 330 J044054.94-0822221|[VV98c] J044053.3-082228|PSCz Q04385-0828|IRAS 0438-084P11|[VV2010c] J044055.0-082222|Gaia DR2 3186063094440691200,*|AGN|G|GiG|IR|Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=70.2290037068%20-8.37281297812&-c.rs=5,541.

In [7]:
NeIR_QSO_obj_t = Table.read(path_raw+'Obj_QSO_Ne-IR_'+date_sample_2+'.vot',format='votable')
NeIR_QSO_obj_t.show_in_notebook()

idx,Main_Id,Id_used_NED,RA,DEC,otype_txt,redshift,rvz_type,ids,otypes,cds_url,SMB_sample_index,NED_Ne_IR_info
0,QSO B1549-790,1Jy 1549-79,239.24529081354166,-79.2345227011389,QSO,0.15,z,Gaia DR3 5778578162594526976|TIC 383941961|1Jy 1549-79|1Jy 1549-790|IERS B1549-790|IRAS 15494-7905|IRAS F15494-7905|LEDA 2828606|MRC 1549-790|PKS 1549-79|PKS 1549-790|PKS J1557-7913|PMN J1556-7914|QSO B1549-790|RORF 1549-790|SUMSS J155657-791403|Gaia DR1 5778578158295248768|VSOP J1556-7914|[CH94] 1549-790|[VV2006c] J155658.5-791405|WMAP J1556-7912|6dFGS gJ155658.9-791404|ICRF J155658.8-791404|[VV2000c] J155658.5-791405|[VV2003c] J155658.5-791405|[VV98c] J155658.5-791405|PSCz Q15494-7905|[MGL2009] BZU J1556-7914|[VV2010c] J155658.5-791405|[MEM2008] 211|AT20G J155658.7-791404|AT20G J155658-791404|[MGL2009] BZQ J1556-7914|PBC J1557.6-7913|[MML2015] 5BZQ J1556-7914|IVS B1549-790|2MASS J15565889-7914042|WMAP J1555-7912|ATPMN J155658.7-791403|SWIFT J1557.8-7913|2MAXI J1555-793|WISE J155658.87-791404.2|3XMM J155658.9-791403|Gaia DR2 5778578162594526976|WISEA J155658.87-791404.2,*|AGN|Bla|G|IR|QSO|Rad|Sy1|Sy2|X|cm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=239.24529081354166%20-79.2345227011389&-c.rs=5,3191.0,NeII|NeIII|NeV
1,3C 48,3C 48,24.42208192967,33.15974455245,QSO,0.369,z,Gaia DR3 316083930556737152|TIC 61422191|2E 444|1Jy 0134+329|1Jy 0134+32|2MASS J01374130+3309350|2MASSI J0137413+330935|3C 48.0|3C 48|3CR 48|4C 32.08|B2 0134+32|B2.1 0134+32|BWE 0134+3254|CTA 15|DA 54|EQ J0137+331|IERS B0134+329|IRAS 01348+3254|IRAS F01348+3254|LEDA 73991|NRAO 79|OHIO C 358|PG 0134+329|PKS 0134+329|QSO B0134+3254|QSO J0137+3309|QSO B0134+329|QSO B0134+32|RORF 0134+329|S2 0134+32|Gaia DR1 316083926261905664|UVQS J013741.30+330935.0|VSOP J0137+3309|WB 0134+3254|[A86] 0134+329|[D71] 0134+32|[DGW65] 8|[DML87] 586|[GR92] 0134+329|[HB93] 0134+329|[S77] 37|[SJS95] 0134+329|[VYP98] 0134+329|[WTW94] 0134+329|1RXS J013741.7+330931|2E 0134.8+3254|87GB 013448.6+325430|ICRF J013741.2+330935|RGB J0137.7+3309|RX J0137.6+3309|[VV2000] J013741.3+330935|[VV2003] J013741.3+330935|[VV2006] J013741.3+330935|[VV96] J013741.3+330935|[VV98] J013741.3+330935|PSCz Q01348+3254|[VV2010] J013741.3+330935|WISE J013741.29+330935.2|NVSS J013741+330935|Cul 0134+328|MITG J013742+3309|TXS 0134+329|WN B0134.8+3254|MY 013449.5+330945.8|RGB J0137+331|[CAB95] IRAS F01348+3254|[P82] 0134+329|[FA87] 0134+329|GB6 B0134+3254|LAMOST J013741.30+330935.1|WISE J013741.30+330935.0|SDSS J013741.30+330935.0|Gaia DR2 316083930556737152|LAMOST J013741.30+330935.0|WISEA J013741.30+330935.0,*|AGN|G|IR|QSO|Rad|UV|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=24.42208192967%2033.15974455245&-c.rs=5,4434.0,NeII
2,2MASS J17193331+5927430,2MASS J17193331+5927430,259.8888333333333,59.46194444444445,QSO,0.13911,z,TIC 462637457|SSTXFLS24 J171933.3+592742|[MFA2007] 91|2MASS J17193331+5927430|FLSVLA J171933.3+592742|SDSS J171933.37+592742.8|[MGC2004] J171933.349+592743.26|FLSGMRT J171933.3+592742|[SW2009] SST24 J171933.40+592743.2|[PCE2006] 818|5MUSES 315|5MUSES J171933.37+592742.8,G|IR|QSO|Rad|SBG,http://vizier.u-strasbg.fr/viz-bin/sed?-c=259.8888333333333%2059.46194444444445&-c.rs=5,5494.0,NeIII
3,3C 263.1,3C 263.1,175.854514,22.115579,QSO,0.366,z,3C 263.1|3CR 263.1|4C 22.30|7C 1140+2223|B2 1140+22|B2.4 1140+22|BWE 1140+2223|Cul 1140+223|DA 306|DB 81|EQ 1140+22|GB6 B1140+2223|LEDA 2819740|MITG J114324+2206|NRAO 383|NVSS J114325+220656|OHIO M 267|PKS J1143+2206|PKS 1140+22|PKS 1140+223|QSO B1140+223|S3 1140+22|VRO 22.11.01|WB 1140+2223|WK 225|[HB91] 1140+223|[GW2008] CoNFIG 107|87GB 114048.6+222345|[MMD97] 114049.26+222334.6|VLSS J1143.4+2206|SDSS J114325.08+220656.0,AGN|G|QSO|Rad,http://vizier.u-strasbg.fr/viz-bin/sed?-c=175.854514%2022.115579&-c.rs=5,5849.0,NeII|NeIII
4,QSO B0125-0635,GSC 04684-01057,21.89802039348,-6.326509127179999,QSO,0.005,z,QSO J0127-0619|Gaia DR3 2478963455764395136|GSC 04684-01057|IRAS F01250-0635|LEDA 5441|Mrk 996|NPM1G -06.0062|PB 8849|QSO B0125-0635|[BDW2002] q0125-0635|2MASX J01273549-0619359|CAI

## Downloading the CDS Tables:

### Seyfert:

In [8]:
start_cds = time.time()
for i, id_ in enumerate(NeIR_Sy_obj_t['Main_Id']):
    print(i,' ',id_,' getting CDS Phot Table')
    cds_req = photCDS(NeIR_Sy_obj_t['cds_url'][i],id_,path_cds_vot)
    print(id_,' CDS VOTable saved OK')
    cds_table = add_bibcode_cds(id_,path_cds_vot)
    cds_table.write(path_phot_cds+id_+'_CDS_phot_tables'+'.vot', #Save the phot table in a vot table file
                         format='votable',overwrite=True)
    print(id_,' CDS Phot Table saved OK\n')
end_cds = time.time()

print('\nExecution Time(seg): ' + str(end_cds - start_cds))
print('Execution Time(min): ' + str((end_cds - start_cds)/60))
print('Execution Time(hrs): ' + str((end_cds - start_cds)/3600))

0   3C 234.0  getting CDS Phot Table
Votable checked
3C 234.0  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  3C 234.0  votable
3C 234.0  CDS Phot Table saved OK

1   2MASS J13000535+1632148  getting CDS Phot Table
Votable checked
2MASS J13000535+1632148  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  2MASS J13000535+1632148  votable
2MASS J13000535+1632148  CDS Phot Table saved OK

2   NGC   262  getting CDS Phot Table
Votable checked
NGC   262  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC   262  votable
NGC   262  CDS Phot Table saved OK

3   IRAS 04385-0828  getting CDS Phot Table
Error in votable, deleting content-length
Votable checked
Votable checked
IRAS 04385-0828  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  IRAS 04385-0828  votable
IRAS 04385-0828  CDS Phot Table saved OK

4   NGC  4945  getting CDS Phot Table
Votable checked
NGC  4945  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  4945  

Adding Bibcode to  NGC  7410  votable
NGC  7410  CDS Phot Table saved OK

43   Mrk  352  getting CDS Phot Table
Votable checked
Mrk  352  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk  352  votable
Mrk  352  CDS Phot Table saved OK

44   LEDA 1365424  getting CDS Phot Table
Votable checked
LEDA 1365424  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  LEDA 1365424  votable
LEDA 1365424  CDS Phot Table saved OK

45   LEDA   27453  getting CDS Phot Table
Votable checked
LEDA   27453  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  LEDA   27453  votable
LEDA   27453  CDS Phot Table saved OK

46   NAME Cyg A  getting CDS Phot Table
Votable checked
NAME Cyg A  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NAME Cyg A  votable
NAME Cyg A  CDS Phot Table saved OK

47   Mrk  877  getting CDS Phot Table
Votable checked
Mrk  877  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk  877  votable
Mrk  877  CDS Phot Table s

Adding Bibcode to  2MASS J08105865+7602424  votable
2MASS J08105865+7602424  CDS Phot Table saved OK

85   IRAS 15091-2107  getting CDS Phot Table
Votable checked
IRAS 15091-2107  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  IRAS 15091-2107  votable
IRAS 15091-2107  CDS Phot Table saved OK

86   UGC 12282  getting CDS Phot Table
Votable checked
UGC 12282  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  UGC 12282  votable
UGC 12282  CDS Phot Table saved OK

87   2MASS J14294306+4747262  getting CDS Phot Table
Votable checked
2MASS J14294306+4747262  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  2MASS J14294306+4747262  votable
2MASS J14294306+4747262  CDS Phot Table saved OK

88   MCG+10-17-021  getting CDS Phot Table
Votable checked
MCG+10-17-021  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  MCG+10-17-021  votable
MCG+10-17-021  CDS Phot Table saved OK

89   ESO 113-45  getting CDS Phot Table
Votable checked
ESO 113-45  CD

Votable checked
Ton  182  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Ton  182  votable
Ton  182  CDS Phot Table saved OK

128   MCG+13-07-002  getting CDS Phot Table
Votable checked
MCG+13-07-002  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  MCG+13-07-002  votable
MCG+13-07-002  CDS Phot Table saved OK

129   NGC  5548  getting CDS Phot Table
Votable checked
NGC  5548  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  5548  votable
NGC  5548  CDS Phot Table saved OK

130   ICRF J145907.5+714019  getting CDS Phot Table
Votable checked
ICRF J145907.5+714019  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  ICRF J145907.5+714019  votable
ICRF J145907.5+714019  CDS Phot Table saved OK

131   NGC   424  getting CDS Phot Table
Votable checked
NGC   424  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC   424  votable
NGC   424  CDS Phot Table saved OK

132   Mrk  509  getting CDS Phot Table
Votable checked
Mrk 

Error in votable, deleting content-length
Votable checked
Votable checked
LEDA   52314  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  LEDA   52314  votable
LEDA   52314  CDS Phot Table saved OK

171   NGC   931  getting CDS Phot Table
Votable checked
NGC   931  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC   931  votable
NGC   931  CDS Phot Table saved OK

172   2MASS J13234951+6541480  getting CDS Phot Table
Votable checked
2MASS J13234951+6541480  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  2MASS J13234951+6541480  votable
2MASS J13234951+6541480  CDS Phot Table saved OK

173   Mrk  335  getting CDS Phot Table
Votable checked
Mrk  335  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk  335  votable
Mrk  335  CDS Phot Table saved OK

174   NGC  3976  getting CDS Phot Table
Votable checked
NGC  3976  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  3976  votable
NGC  3976  CDS Phot Table saved OK

1

Adding Bibcode to  Mrk  231  votable
Mrk  231  CDS Phot Table saved OK

212   Mrk   79  getting CDS Phot Table
Votable checked
Mrk   79  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk   79  votable
Mrk   79  CDS Phot Table saved OK

213   LEDA   68751  getting CDS Phot Table
Votable checked
LEDA   68751  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  LEDA   68751  votable
LEDA   68751  CDS Phot Table saved OK

214   NGC  7319  getting CDS Phot Table
Votable checked
NGC  7319  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  7319  votable
NGC  7319  CDS Phot Table saved OK

215   Mrk  863  getting CDS Phot Table
Votable checked
Mrk  863  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk  863  votable
Mrk  863  CDS Phot Table saved OK

216   2MASX J18334627+4727027  getting CDS Phot Table
Votable checked
2MASX J18334627+4727027  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  2MASX J18334627+4727027  votabl

Votable checked
ESO  33-2  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  ESO  33-2  votable
ESO  33-2  CDS Phot Table saved OK

255   NGC  4725  getting CDS Phot Table
Votable checked
NGC  4725  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  4725  votable
NGC  4725  CDS Phot Table saved OK

256   NGC  7682  getting CDS Phot Table
Votable checked
NGC  7682  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  7682  votable
NGC  7682  CDS Phot Table saved OK

257   [VV2003c] J124819.8+662143  getting CDS Phot Table
Votable checked
[VV2003c] J124819.8+662143  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  [VV2003c] J124819.8+662143  votable
[VV2003c] J124819.8+662143  CDS Phot Table saved OK

258   7C 1821+6419  getting CDS Phot Table
Votable checked
7C 1821+6419  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  7C 1821+6419  votable
7C 1821+6419  CDS Phot Table saved OK

259   MCG-01-24-012  getting CDS Phot Ta

Votable checked
Mrk  478  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk  478  votable
Mrk  478  CDS Phot Table saved OK

297   IRAS 07598+6508  getting CDS Phot Table
Votable checked
IRAS 07598+6508  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  IRAS 07598+6508  votable
IRAS 07598+6508  CDS Phot Table saved OK

298   2MASS J10434845+5811477  getting CDS Phot Table
Votable checked
2MASS J10434845+5811477  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  2MASS J10434845+5811477  votable
2MASS J10434845+5811477  CDS Phot Table saved OK

299   NGC  3081  getting CDS Phot Table
Votable checked
NGC  3081  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  3081  votable
NGC  3081  CDS Phot Table saved OK

300   NGC  4507  getting CDS Phot Table
Error in votable, deleting content-length
Error in votable, deleting content-length
Votable checked
Votable checked
Votable checked
NGC  4507  CDS VOTable saved OK
Searching for Bibcode
A

### QSO:

In [9]:
start_cds = time.time()
for i, id_ in enumerate(NeIR_QSO_obj_t['Main_Id']):
    print(i,' ',id_,' getting CDS Phot Table')
    cds_req = photCDS(NeIR_QSO_obj_t['cds_url'][i],id_,path_cds_vot)
    print(id_,' CDS VOTable saved OK')
    cds_table = add_bibcode_cds(id_,path_cds_vot)
    cds_table.write(path_phot_cds+id_+'_CDS_phot_tables'+'.vot', #Save the phot table in a vot table file
                         format='votable',overwrite=True)
    print(id_,' CDS Phot Table saved OK\n')
end_cds = time.time()

print('\nExecution Time(seg): ' + str(end_cds - start_cds))
print('Execution Time(min): ' + str((end_cds - start_cds)/60))
print('Execution Time(hrs): ' + str((end_cds - start_cds)/3600))

0   QSO B1549-790  getting CDS Phot Table
Votable checked
QSO B1549-790  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  QSO B1549-790  votable
QSO B1549-790  CDS Phot Table saved OK

1   3C  48  getting CDS Phot Table
Votable checked
3C  48  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  3C  48  votable
3C  48  CDS Phot Table saved OK

2   2MASS J17193331+5927430  getting CDS Phot Table
Votable checked
2MASS J17193331+5927430  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  2MASS J17193331+5927430  votable
2MASS J17193331+5927430  CDS Phot Table saved OK

3   3C 263.1  getting CDS Phot Table
Votable checked
3C 263.1  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  3C 263.1  votable
3C 263.1  CDS Phot Table saved OK

4   QSO B0125-0635  getting CDS Phot Table
Votable checked
QSO B0125-0635  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  QSO B0125-0635  votable
QSO B0125-0635  CDS Phot Table saved OK

5   2MASS J234

### Notebook Info:

In [10]:
%load_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.12.0

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.15.0-72-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 5.1

requests: 2.29.0
astropy : 5.1
sys     : 3.9.12 (main, Apr  5 2022, 06:56:58) 
[GCC 7.5.0]
numpy   : 1.21.5

Watermark: 2.3.1

